## Пример на pyspark

В качестве набора данных для примера будем использовать данные конкурса про ответы студентов на тесты
https://www.kaggle.com/c/riiid-test-answer-prediction

При подключении к spark драйверу установим лимиты по памяти и по числу ядер. Также выберем номер порта для Spark UI

Нужно выбрать уникальное имя приложения и номер порта, чтобы не войти в коллизию с другими пользователями

In [1]:
%pylab inline
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS-PySpark-Notebook")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)

Данные будем читать из заранее сконвертированного parquet

In [4]:
df = spark.read.parquet("data/train.parquet")

Схема данных и первые 10 записей

In [6]:
df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- content_id: integer (nullable = true)
 |-- content_type_id: integer (nullable = true)
 |-- task_container_id: integer (nullable = true)
 |-- user_answer: integer (nullable = true)
 |-- answered_correctly: integer (nullable = true)
 |-- prior_question_elapsed_time: double (nullable = true)
 |-- prior_question_had_explanation: boolean (nullable = true)



In [7]:
df.show(10)

+--------+-----------+---------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|  row_id|  timestamp|  user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+--------+-----------+---------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|10059541|  953601099|218331526|      1114|              0|              650|          3|                 1|                    17000.0|                          true|
| 9882908|26098315317|214467360|       721|              0|              375|          1|                 1|                    15000.0|                          true|
|11032662|  590935189|239834028|       231|              0|              277|          1|                 1|                    18000.0|                        

In [8]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter
df

row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
10059541,953601099,218331526,1114,0,650,3,1,17000.0,true
9882908,26098315317,214467360,721,0,375,1,1,15000.0,true
11032662,590935189,239834028,231,0,277,1,1,18000.0,true
11487463,13117283988,249140958,7219,0,15,0,0,31750.0,false
10193731,178006028,220767027,4144,0,304,0,1,13000.0,true
11451805,1054272891,248526815,9593,0,307,0,0,14000.0,true
9751638,4175717773,212002144,8161,0,4604,2,0,44000.0,true
11102000,26201536504,241050300,8014,0,316,0,1,48000.0,true
11063074,1030127,240427856,609,0,21,1,1,18000.0,true
11747704,51839059,254609209,665,0,42,1,1,15000.0,true


Замерим время выполнения простых запросов с группировками

In [9]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col

In [10]:
%%time
(
    df
        .select('content_id', 'answered_correctly')
        .groupBy('content_id')
        .mean('answered_correctly')
        .show()
)

+----------+-----------------------+
|content_id|avg(answered_correctly)|
+----------+-----------------------+
|     10817|     0.7436510307738273|
|      1591|     0.8129474940334129|
|      6336|      0.777745995423341|
|      9465|     0.4461902348369657|
|     11141|     0.8700970054829186|
|       833|     0.9050215571899569|
|      5156|    0.47105625162657544|
|      4818|     0.4098718947459534|
|      1238|      0.597189012655275|
|      2366|     0.5468291250733999|
|      3175|     0.8605798889574336|
|      6397|     0.7980720446473871|
|      3749|     0.6274719401389631|
|      1829|     0.7589958158995815|
|     11748|     0.9651639344262295|
|     11858|     0.6889063317634746|
|       496|     0.7542558870632896|
|     11458|     0.7225150602409639|
|      3997|     0.6283098243009156|
|      7554|     0.8941355674028941|
+----------+-----------------------+
only showing top 20 rows

CPU times: user 47 µs, sys: 7.42 ms, total: 7.46 ms
Wall time: 12 s


In [11]:
%%time
(
    df
        .select('user_id', 'answered_correctly')
        .where(col('answered_correctly') != -1)
        .groupby('user_id')
        .mean('answered_correctly')
        .show()
)

+---------+-----------------------+
|  user_id|avg(answered_correctly)|
+---------+-----------------------+
|252345392|       0.56480117820324|
|253500385|     0.5285296981499513|
|242039738|    0.46367041198501874|
|254408119|     0.7657534246575343|
|224426519|     0.7173333333333334|
|240485154|     0.5521978021978022|
|244175802|     0.7359550561797753|
|211646563|     0.7669491525423728|
|219176829|     0.5384615384615384|
|218611655|      0.676829268292683|
|212559006|                  0.525|
|259875659|     0.6183574879227053|
|240750690|     0.6890756302521008|
|245081718|    0.30612244897959184|
|254957588|     0.2702702702702703|
|226534187|     0.6363636363636364|
|257509511|    0.49019607843137253|
|220663840|     0.7661870503597122|
|252861630|               0.546875|
|243302977|     0.6267942583732058|
+---------+-----------------------+
only showing top 20 rows

CPU times: user 7.6 ms, sys: 0 ns, total: 7.6 ms
Wall time: 13.6 s


## Упражнение 1
Выведите top 10 студентов с наилучшими результатами. 
Обратите внимание, что поле answered_correctly равно -1, если это была лекция, а не тест. Такие записи нужно исключить.

## Упражнение 2
Выведите top 10 задач с наихудшими результатами

## pyspark user defined functions (UDF)

Как и для других языков, поддерживаемых Spark, для python есть возможность использовать UDF. При этом возникают дополнительные накладные расходы по сравнению с Java и Scala.

In [12]:
from pyspark.sql.types import LongType

def to_months(ms):
    return ms // 31536000000 // 12 #1 year = 31536000000 ms

to_months_udf = f.udf(to_months, LongType())

Замерим время выполнения без UDF

In [13]:
%%time
(
    df
        .select("content_id", "timestamp")
        .groupby("content_id")
        .mean("timestamp")
        .show()
)

+----------+--------------------+
|content_id|      avg(timestamp)|
+----------+--------------------+
|      4818| 6.142197065523717E9|
|      7993| 8.863857741624483E9|
|      4519| 5.839083879095016E9|
|      1088| 9.316805237791412E9|
|      6397| 6.173872191169965E9|
|      5518| 7.962030146793127E9|
|      9427| 8.169934790387074E9|
|       148| 6.778016709578808E9|
|      2366| 9.180255834014534E9|
|      3749| 6.928784635868185E9|
|      9465| 9.754291493026693E9|
|      3794|   7.4190074291877E9|
|      7240|1.012949297380670...|
|      1580| 8.499988982626443E9|
|     23336| 7.288716371091926E9|
|      6466| 8.827348933138468E9|
|      6654|1.130907472662850...|
|      6620| 8.362320610059411E9|
|       496| 7.627430069740495E9|
|     10817|1.349889894171885...|
+----------+--------------------+
only showing top 20 rows

CPU times: user 6.01 ms, sys: 0 ns, total: 6.01 ms
Wall time: 6.52 s


Применим простой UDF к похожему запросу

In [17]:
%%time
(
    df
        .select("content_id", to_months_udf("timestamp").alias("months"))
        .groupBy("content_id")
        .mean("months")
        .show()
)

+----------+-----------+
|content_id|avg(months)|
+----------+-----------+
|      1342|        0.0|
|      1238|        0.0|
|      5156|        0.0|
|      4519|        0.0|
|      3918|        0.0|
|       148|        0.0|
|       471|        0.0|
|      3175|        0.0|
|      6397|        0.0|
|     23336|        0.0|
|      4818|        0.0|
|      6336|        0.0|
|      7880|        0.0|
|      3749|        0.0|
|      6620|        0.0|
|       496|        0.0|
|      9900|        0.0|
|      5518|        0.0|
|      2142|        0.0|
|     10206|        0.0|
+----------+-----------+
only showing top 20 rows

CPU times: user 20.9 ms, sys: 1.35 ms, total: 22.3 ms
Wall time: 1min 23s


Перепишем логику, которая была в UDF

In [18]:
%%time
(
    df
        .select("content_id", (col("timestamp") / 31536000000 / 12).alias("months"))
        .groupby("content_id")
        .mean("months")
        .show()
)

+----------+--------------------+
|content_id|         avg(months)|
+----------+--------------------+
|      4818|0.016230649272587203|
|      7993| 0.02342259043004947|
|      4519|0.015429677931821349|
|      1088| 0.02461949633696784|
|      6397| 0.01631435024302904|
|      5518|0.021039526643606057|
|      9427|0.021588911060341286|
|       148|0.017910791660268714|
|      2366|0.024258666904528516|
|      3749|0.018309193291973676|
|      9465|0.025775546182740073|
|      3794|0.019604598525462175|
|      7240|  0.0267670095917013|
|      1580|0.022461073541947936|
|     23336|0.019260306662998706|
|      6466|0.023326116536493923|
|      6654| 0.02988403392585326|
|      6620| 0.02209728725387761|
|       496| 0.02015535174018184|
|     10817|0.035670606454313726|
+----------+--------------------+
only showing top 20 rows

CPU times: user 1.41 ms, sys: 4.24 ms, total: 5.65 ms
Wall time: 6.08 s


## Упражнение 3
Постройте гистограмму по числу месяцев до первого взаимодействия студента с заданием

In [20]:
%%time
df_pandas = (
    df
        .select("content_id", (col("timestamp") / 31536000000 / 12).alias("months"))
        .groupby("content_id")
        .mean("months")
        .toPandas()
)

CPU times: user 52.3 ms, sys: 405 µs, total: 52.8 ms
Wall time: 7.22 s


In [21]:
df_pandas.head()

,content_id,avg(months)
0,10623,0.024414
1,1829,0.024396
2,4101,0.025990
3,4818,0.016231
4,1238,0.019187


## Обогащение данных

Таблица с вопросами лежит в отдельном файле questions.csv. 

In [22]:
questions = spark.read.csv("data/questions.csv", header=True, inferSchema=True)

In [23]:
questions.count()

13523

In [24]:
questions.printSchema()

root
 |-- question_id: integer (nullable = true)
 |-- bundle_id: integer (nullable = true)
 |-- correct_answer: integer (nullable = true)
 |-- part: integer (nullable = true)
 |-- tags: string (nullable = true)



Объединим ее с ответами при условии, что эта запись ссылкается на вопрос если content_type_id и content_id - идентификатор вопроса.

In [25]:
df_join = (
    df
        .where(f.col("content_type_id") == 0)
        .join(questions, df.content_id == questions.question_id, 'left')
)

In [26]:
df_join

row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags
10059541,953601099,218331526,1114,0,650,3,1,17000.0,true,1114,1114,3,2,143 71 29 102
9882908,26098315317,214467360,721,0,375,1,1,15000.0,true,721,721,1,2,143 176 6 38 102
11032662,590935189,239834028,231,0,277,1,1,18000.0,true,231,231,1,2,90 100 92 102
11487463,13117283988,249140958,7219,0,15,0,0,31750.0,false,7219,7216,2,7,50 19 21
10193731,178006028,220767027,4144,0,304,0,1,13000.0,true,4144,4144,0,5,1
11451805,1054272891,248526815,9593,0,307,0,0,14000.0,true,9593,9593,1,5,73
9751638,4175717773,212002144,8161,0,4604,2,0,44000.0,true,8161,8159,1,7,39 42 160 135 162
11102000,26201536504,241050300,8014,0,316,0,1,48000.0,true,8014,8014,0,7,98 18 46 135
11063074,1030127,240427856,609,0,21,1,1,18000.0,true,609,609,1,2,62 138 41 38 92
11747704,51839059,254609209,665,0,42,1,1,15000.0,true,665,665,1,2,143 114 38 81


Проверим, что вероястность правильного ответа не зависит от его номера.

In [27]:
%%time
(
    df_join
        .select('correct_answer', 'answered_correctly')
        .groupby('correct_answer')
        .mean('answered_correctly')
        .show()
)

+--------------+-----------------------+
|correct_answer|avg(answered_correctly)|
+--------------+-----------------------+
|             1|     0.6728072675886204|
|             3|     0.6592246990371898|
|             2|     0.6192199974012494|
|             0|     0.6674206042887552|
+--------------+-----------------------+

CPU times: user 4.99 ms, sys: 517 µs, total: 5.5 ms
Wall time: 10.8 s


In [28]:
df_join.toPandas().head()

KeyboardInterrupt: 

## Упражнение 4

В файле "lectures.csv" хранится информация об лекциях. Объедините эту таблицу с основным набором данных при условии, что content_type_id == 1.